In [1]:
import os 
import tqdm
import math
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from numpy import count_nonzero

dir_ = '../data/'
group_dir_ = '../data/groups/high'
folder = 'all (min-max)(user)'
file_name = 'normalized_minmax_filter_track_5_user_100.pkl'

In [2]:
predictions = pd.read_pickle(os.path.join(dir_, folder, 'topN_pred_' + file_name[:-3] + 'pkl'))
test = pd.read_pickle(os.path.join(dir_, folder, 'test_' + file_name[:-3] + 'pkl'))
test.sort_values(by=['uid','tid'])[:5]

,uid,tid,rating,count
26,0,9,1.071429,15
37,0,15,0.214286,3
45,0,22,0.428571,6
64,0,28,0.285714,4
75,0,33,0.071429,1


In [3]:
predictions.sort_values(by=['rating'],  ascending=False)

,uid,tid,rating
134345540,867,70,1.228373
126460659,816,70,1.224137
39397515,254,70,1.216006
108520730,700,70,1.205024
32406146,209,70,1.203970
...,...,...,...
111218542,717,56485,0.007406
125183091,807,120430,0.007093
125153567,807,90904,0.006055
111285284,717,124033,0.005310


In [4]:
groups_n =[]
group_sizes = ['2', '3', '4', '5', '6', '7', '8', '9', '10']
for i in range(len(group_sizes)):
    groups = np.loadtxt(os.path.join(group_dir_, group_sizes[i] + '.csv'), delimiter=',')
    groups_n.append(groups)
    print(len(groups[0]))

2
3
4
5
6
7
8
9
10


In [5]:
num_user = len(test['uid'].unique())
num_user

953

In [6]:
# Personal Recommendation
n=20
top_n_lists = []

for i in tqdm(range(num_user)):
    prediction = predictions[predictions['uid'] == i]
    prediction = prediction.sort_values(by=['rating'],  ascending=False)
    prediction = prediction[:n]
#     print(prediction)
    top_n_list = []
    for _, row in prediction.iterrows():
        top_n_list.append(row[1])
#     print(top_n_list)
    top_n_lists.append(top_n_list)

In [7]:
evaluation = []
satisfication = 0
for i in tqdm(range(num_user)):
    top_n_list = top_n_lists[i]
    precision = 0
    for j in top_n_list:
        p = test[test['uid'] == i]
        p = p[p['tid']==j]
        if len(p) > 0:
            precision += 1
    satisfication += precision / n
    evaluation.append(precision)
#     print(precision, satisfication)
print(satisfication/num_user)


0.20225603357817432


In [8]:
precision_4 = []
precision_20 = []
precision_50 = []

recall_4 = []
recall_20 = []
recall_50 = []

In [9]:
# Precision and Recall
top_n_size = 50
top_n_lists_g = []
for groups in tqdm(groups_n):

    group_size = len(groups[0])
    top_n_lists = []

    for group in groups:
        rating_table = pd.DataFrame() 
        for member in group:
            prediction = predictions[predictions['uid'] == member].copy()
            if len(rating_table) == 0:
                rating_table = prediction
            else:
                rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
        rating_table['rating'] /= group_size 

        rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
        rating_table = rating_table[:top_n_size]
        top_n_list = []
        for _, row in rating_table.iterrows():
            top_n_list.append(row[0])
        top_n_lists.append(top_n_list)
    top_n_lists_g.append(top_n_lists)   

threshold = 0.0
for top_n_size in tqdm([50, 20, 4]):
    avg_precisions = []
    avg_recalls = []
    for i in range(len(groups_n)):
        groups = groups_n[i]
        top_n_lists = top_n_lists_g[i]

        precisions = []
        recalls =[]
        for j in (range(len(groups))):

            group = groups[j]
            top_n_list = top_n_lists[j][:top_n_size]
            
            
            for k in range(len(group)):
                high_rating = 0
                for l in range(top_n_size):
                    uid = group[k]
                    tid = top_n_list[l]
                    t = test[test['uid'] == uid]
                    t = t[t['tid'] == tid]
                    if len(t) > 0 and t.iloc[0]['rating'] > threshold:
                        high_rating += 1
                        
                precision = high_rating / top_n_size
                recall = high_rating / len(test[test['uid'] == uid])
                precisions.append(precision)
                recalls.append(recall)
                
        avg_precision = 0
        for precision in precisions:
            avg_precision += precision
        avg_precision /= len(precisions)
        avg_precisions.append(avg_precision)
        
        avg_recall = 0
        for recall in recalls:
            avg_recall += recall
        avg_recall /= len(recalls)
        avg_recalls.append(avg_recall)
        
    if top_n_size == 4:
        precision_4.append(avg_precisions)
    elif top_n_size == 20:
        precision_20.append(avg_precisions)
    elif top_n_size == 50:
        precision_50.append(avg_precisions)
        
    if top_n_size == 4:
        recall_4.append(avg_recalls)
    elif top_n_size == 20:
        recall_20.append(avg_recalls)
    elif top_n_size == 50:
        recall_50.append(avg_recalls)


In [10]:
for i in precision_4:
    print(i)

for i in precision_20:
    print(i)

for i in precision_50:
    print(i)

for i in recall_4:
    print(i)

for i in recall_20:
    print(i)

for i in recall_50:
    print(i)

[0.21686746987951808, 0.20390070921985815, 0.15086206896551724, 0.16136363636363638, 0.15833333333333333, 0.15773809523809523, 0.2, 0.1736111111111111, 0.17916666666666667]
[0.16385542168674713, 0.1432624113475178, 0.12758620689655173, 0.12681818181818189, 0.11222222222222222, 0.11130952380952379, 0.11125, 0.1111111111111111, 0.12166666666666666]
[0.13289156626506013, 0.12170212765957435, 0.11172413793103442, 0.1019999999999999, 0.10155555555555543, 0.09190476190476188, 0.09099999999999994, 0.08944444444444438, 0.08966666666666664]
[0.001619628797467798, 0.0013996793894849335, 0.0011046156813401963, 0.0013139792422374392, 0.0014316765635236036, 0.0010447168584341077, 0.0016220712150045434, 0.0017167053756327823, 0.0016531569974731185]
[0.006098042071674393, 0.005601658446345695, 0.0051142732803318765, 0.005248144130521653, 0.004916521993972821, 0.005429053239236008, 0.004564844365175286, 0.005071432975940561, 0.00497440735595704]
[0.012342961690035697, 0.011212453927541449, 0.010800204

In [11]:
def dcg_at_k(r, k, method=0):
    r = np.asfarray(r)[:k]
    for i in range(len(r)):
        r[i] = 2**r[i] -1
    
    if r.size:
        if method == 0:
            return r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
        elif method == 1:
            return np.sum(r / np.log2(np.arange(2, r.size + 2)))
        else:
            raise ValueError('method must be 0 or 1.')
    return 0.


def ndcg_at_k(r, r_max, k, method=0):
    dcg_max = dcg_at_k(r_max, k, method)
    if not dcg_max:
        return 0.
    return dcg_at_k(r, k, method) / dcg_max

In [12]:
nDCG_4 = []
nDCG_20 = []
nDCG_50 = []

In [13]:
# Precision and Recall
top_n_size = 50
top_n_lists_g = []
for groups in tqdm(groups_n):

    group_size = len(groups[0])
    top_n_lists = []

    for group in groups:
        rating_table = pd.DataFrame() 
        for member in group:
            prediction = predictions[predictions['uid'] == member].copy()
            if len(rating_table) == 0:
                rating_table = prediction
            else:
                rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
        rating_table['rating'] /= group_size 

        rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
        rating_table = rating_table[:top_n_size]
        top_n_list = []
        for _, row in rating_table.iterrows():
            top_n_list.append(row[0])
        top_n_lists.append(top_n_list)
    top_n_lists_g.append(top_n_lists)   

threshold = 0.0
for top_n_size in tqdm([50, 20, 4]):
    avg_nDCGs = []
    for i in range(len(groups_n)):
        groups = groups_n[i]
        top_n_lists = top_n_lists_g[i]

        nDCGs = []
        for j in (range(len(groups))):

            group = groups[j]
            top_n_list = top_n_lists[j][:top_n_size]
            
            
            for k in range(len(group)):
                truth_rating = []
                uid = group[k]
                for l in range(top_n_size):
                    tid = top_n_list[l]
                    t = test[test['uid'] == uid]
                    t = t[t['tid'] == tid]
                    
                    if len(t) > 0:
                        truth_rating.append(t.iloc[0]['rating'])
                    else:
                        truth_rating.append(0)
                max_rating = test[test['uid']==uid].sort_values(by=['rating'],  ascending=False)['rating'].values[:top_n_size]
                nDCG = ndcg_at_k(truth_rating, max_rating, top_n_size, method=1)
                nDCGs.append(nDCG)                
        avg_nDCG = 0
        for nDCG in nDCGs:
            avg_nDCG += nDCG
        avg_nDCG /= len(nDCGs)
        avg_nDCGs.append(avg_nDCG)
        
    if top_n_size == 4:
        nDCG_4.append(avg_nDCGs)
    elif top_n_size == 20:
        nDCG_20.append(avg_nDCGs)
    elif top_n_size == 50:
        nDCG_50.append(avg_nDCGs)

In [14]:
for i in nDCG_4:
    print(i)

for i in nDCG_20:
    print(i)

for i in nDCG_50:
    print(i)

[0.040655660483195855, 0.03476636527614594, 0.016865717991292255, 0.014158872190144417, 0.02751263391668501, 0.01660530253633658, 0.01515080658975465, 0.02591798843366751, 0.029444815924393828]
[0.04192347755081453, 0.036339572729193526, 0.021523777897237667, 0.020049073990919705, 0.025533537999912388, 0.021141075070637356, 0.01835380404351632, 0.030351874724514456, 0.03240150153605157]
[0.04504232141707047, 0.042235875968323566, 0.026408645268837277, 0.023333905109691903, 0.029130592394559143, 0.026545558163159743, 0.02056760636208795, 0.03137302432604147, 0.03363735020118452]
